<h1 style="color:#8B66FF;">Question 3</h1>
<h3 style="color:#659FFF;">implementation of patch match algorithm for image completion(image painting)
</h3>

I used patch match method described in <a href="https://gfx.cs.princeton.edu/pubs/Barnes_2009_PAR/index.php">here</a>.
Suppose that the masked area which is intended to be removed is stored in array A. Moreover, the
reference area which is used to fill the missing parts is stored in array B. In order to use B as a reference, we store the
corresponding indices in an array named offset. offset[j, i, 0] and offset[j, i, 1] show the
row and column in B where we use the pixel in those coordinates to indicate the best match for pixel (j, i)
in A (naturally, this offset matrix gets updated in each step to result in a better output).
The general steps of this algorithm are as follows:
<ol>
  <li>Initialization</li>
  <li>Propagation</li>
  <li>Random Search</li>
</ol>

In this question I also used an image pyramid to calculate the offset first in the image when it is scaled down to the powers of two.
Then continued by using the output result of the offset for the next depth where finally we reach to depth = 0 which is the actual size of the image.

<br>
The inpaint method gets inputs image, the coordinates where A starts and ends and the coordinates where B(as our refrence) starts and ends in the image.


In [ ]:
import numpy as np
import cv2

def inpaint(img, mask_x0, mask_y0, mask_x1, mask_y1, ref_x0, ref_y0, ref_x1, ref_y1):
    depth = 3
    # building image pyramid here:
    for i in range(depth, -1, -1):
        image = cv2.resize(img, (0, 0), fx=0.5 ** i, fy=0.5 ** i)
        mx0, my0, mx1, my1, rx0, ry0, rx1, ry1 = convertToDepth(mask_x0, mask_y0, mask_x1, mask_y1, ref_x0, ref_y0,
                                                                ref_x1, ref_y1, i)
        A = image[my0:my1, mx0:mx1, :]
        A_h, A_w, _ = A.shape
        B = image[ry0: ry1, rx0: rx1, :] # B[j, i, :] corresponds to the best fit to get substituted in pixel (j, i) in A.
        if i == depth:
            offset = np.random.randint(A_h, B.shape[0] - A_h, (A_h, A_w, 2))
        else: # use the previous results for the next depth
            offset = np.repeat(offset, repeats=2, axis=0)
            offset = np.repeat(offset, repeats=2, axis=1)
            offset = offset * 2
        iterate(image, A, B, offset, mx0, my0)
        cv2.imwrite('../results/res3/depth'+str(i)+'.jpg', image)
    img = np.copy(image)


def convertToDepth(mask_x0, mask_y0, mask_x1, mask_y1, ref_x0, ref_y0, ref_x1, ref_y1, i):
    mask_x0, mask_y0 = int(mask_x0 * 0.5 ** i), int(mask_y0 * 0.5 ** i)
    mask_x1, mask_y1 = int(mask_x1 * 0.5 ** i), int(mask_y1 * 0.5 ** i)
    ref_x0, ref_y0 = int(ref_x0 * 0.5 ** i), int(ref_y0 * 0.5 ** i)
    ref_x1, ref_y1 = int(ref_x1 * 0.5 ** i), int(ref_y1 * 0.5 ** i)
    return mask_x0, mask_y0, mask_x1, mask_y1, ref_x0, ref_y0, ref_x1, ref_y1

<h3 style="color:#659FFF;">Initialization</h3>
A can be initialized either by assigning random values to the field, or by using prior information. When initializing with random offsets, we use independent uniform samples across the full range of image B.
In the first iteration, we generate random valued with np.random fundtion. Then proceed with the output of each iteration.
 In this case that we used image pyramid, I turned the offset matix in each depth, to a matrix with the double number of rows, columns and values.


<br>

After initialization, we perform an iterative process of improving the offset matrix.
In each iteration, we start from a corner of A and then continue by scanning the pixels and find the patch to the center of that pixel and then by comparing it to other possible options, we can find a better match for that pixel.
In the odd iterations, I start from the top left corner and move my way down to the bottom right corner of the A. I use the opposite direction in even iterations meaning that I start from
the bottom right corner and move my way down to the top left corner.
<h3 style="color:#659FFF;">Propagation</h3>
To be more precise on how I improve the offset[j,i] in the odd iterations, I used the values in offset[j - 1,i] and offset[j,i - 1]
Assuming that the patch offsets are likely to be the same. For example, if there is a good mapping at (j - 1,i), we try to use the translation of that mapping one pixel to the right for our mapping at (j, i).

In [ ]:
def iterate(image, A, B, offset, mask_x0, mask_y0):
    for i in range(1, 9):
        if i % 2 == 1:
            odd_iteration(image, A, B, offset, mask_x0, mask_y0)
        else:
            # image, A, B, offset = np.rot90(image, 2), np.rot90(A, 2), np.rot90(B, 2), np.rot90(offset, 2)
            # odd_iteration(image, A, B, offset,
            #               image.shape[1] - mask_x0 - A.shape[1], image.shape[0] - mask_y0 - A.shape[0], )
            # image, A, B, offset = np.rot90(image, -2), np.rot90(A, -2), np.rot90(B, 2), np.rot90(offset, 2)
            even_iteration(image, A, B, offset, mask_x0, mask_y0)
    cv2.imwrite('../results/res3/iteration' + str(i) + '.jpg', image)

In [ ]:
def SSD(shifted1, shifted2):
    if shifted1.shape != shifted2.shape:
        return float("inf")
    return np.average((shifted1 - shifted2) ** 2)

def getPatchCenteredIn(center_y, center_x, ps_half, arr):
    return arr[center_y - ps_half: center_y + ps_half + 1, center_x - ps_half: center_x + ps_half + 1, :]

def odd_iteration(image, A, B, offset, mask_x0, mask_y0):
    p_half, patch_size = 2, 5
    for j in range(0, offset.shape[0]):
        for i in range(0, offset.shape[1]):
            src_patch = getPatchCenteredIn(j + mask_y0, i + mask_x0, p_half, image)
            rj, ri = offset[j, i, 0], offset[j, i, 1]
            ref_patch = getPatchCenteredIn(rj, ri, p_half, B)

            if j != 0 and i == 0:
                n2j, n2i = offset[j - 1, i, 0], offset[j - 1, i, 1]
                neighbor2 = getPatchCenteredIn(n2j + 1, n2i, p_half, B)
                ref_ssd, n2_ssd = SSD(src_patch, ref_patch), SSD(src_patch, neighbor2)
                min_ssd = min(ref_ssd, n2_ssd)
                if n2_ssd == min_ssd:
                    offset[j, i, 0], offset[j, i, 1] = n2j + 1, n2i
                random_search(B, offset, patch_size, p_half, src_patch, min_ssd, i, j)
            elif j == 0 and i != 0:
                n1j, n1i = offset[j, i - 1, 0], offset[j, i - 1, 1]
                neighbor1 = getPatchCenteredIn(n1j, n1i + 1, p_half, B)
                ref_ssd, n1_ssd = SSD(src_patch, ref_patch), SSD(src_patch, neighbor1)
                min_ssd = min(ref_ssd, n1_ssd)
                if n1_ssd == min_ssd:
                    offset[j, i, 0], offset[j, i, 1] = n1j, n1i + 1
                random_search(B, offset, patch_size, p_half, src_patch, min_ssd, i, j)
            elif j != 0 or i != 0:
                n1j, n1i, n2j, n2i = offset[j, i - 1, 0], offset[j, i - 1, 1], offset[j - 1, i, 0], offset[
                    j - 1, i, 1]
                neighbor1 = getPatchCenteredIn(n1j, n1i + 1, p_half, B)
                neighbor2 = getPatchCenteredIn(n2j + 1, n2i, p_half, B)
                ref_ssd, n1_ssd, n2_ssd = SSD(src_patch, ref_patch), SSD(src_patch, neighbor1), SSD(src_patch,
                                                                                                    neighbor2)
                min_ssd = min(ref_ssd, n1_ssd, n2_ssd)
                if n1_ssd == min_ssd:
                    offset[j, i, 0], offset[j, i, 1] = n1j + 1, n1i
                elif n2_ssd == min_ssd:
                    offset[j, i, 0], offset[j, i, 1] = n2j, n2i + 1
                random_search(B, offset, patch_size, p_half, src_patch, min_ssd, i, j)

    apply_offsets(image, A, B, offset, mask_x0, mask_y0, p_half)

When moving from the top left corner, we intend to compare the patches centered in the targeted area with the patches in B(the patch centered in offset[j,i] in reference) ,
left and top of that pixel. Also note that when we start from top left, and get the neighbors to the left and top of the current pixel, the pixel [0,0] gets ignored (however not in ther even iterations) and also the pixels in the first column only have the top neighbor and similarly, the pixels in the first row only have the left neighbor.
Additionally, in the general case, we have both the neighbors and the patch in reference B. Then continue by computing the ssd value in which the patches are compared to the current value in A area. Finally, teh patch with minimum value of ssd difference gets replaced in offset.
In the case of even iterations, I start from the right bottom and move upwards and consider the patches to the right and bottom of the current patch as its neighbors.

<h3 style="color:#659FFF;">Random Search</h3>
In the random search, I consider a window around my reference and retrieve sample and random patches from it and compare it to the patch in A.
If it was a better match than the previous min value, it gets replaced in the offset array. Then I continue by halving the window size till it is smaller than the patch size.

In [ ]:
def random_search(B, offset, patch_size, ps_half, src_patch, min_ssd, i, j):
    w, alpha = B.shape[0], 1 / 2
    while w > 2 * patch_size + 1:
        x_rand, y_rand = np.random.randint(int(w / 10), w - int(w / 10)), np.random.randint(int(w / 10),
                                                                                            w - int(w / 10))
        random_patch = getPatchCenteredIn(y_rand, x_rand, ps_half, B)
        if SSD(src_patch, random_patch) < min_ssd:
            offset[j, i, 0], offset[j, i, 1] = y_rand, x_rand
        w = np.floor(alpha * w)


I also apply the retrieved offsets in the offset array to the image (area where A is) after each iteration.
It is done by a wighted

In [ ]:
def apply_offsets(image, A, B, offset, mask_x0, mask_y0, p_half):
    for i in range(offset.shape[1]):
        for j in range(offset.shape[0]):
            image[mask_y0 + j, mask_x0 + i, :] = B[offset[j, i, 0], offset[j, i, 1], :]
            # average, num = np.zeros((1, 3), dtype=float), 0
            # if i % 2 == 0:
            #     rand_y, rand_x = random.randint(-p_half, p_half+1), random.randint(-p_half, p_half+1)
            #     if -1 < j - rand_y < offset.shape[0] and -1 < i - rand_x < offset.shape[1]:
            #         y, x = offset[j - rand_y, i - rand_x, 0] + rand_y, offset[j - rand_y, i - rand_x, 1] + rand_x
            #         if -1 < y and x < B.shape[0]:
            #             average = B[y, x, :]
            #             num = 1
            # else:
            #     for k in range(-p_half, p_half + 1):
            #         for l in range(-p_half, p_half + 1):
            #             if -1 < j - l < offset.shape[0] and -1 < i - k < offset.shape[1]:
            #                 y, x = offset[j - l, i - k, 0] + l, offset[j - l, i - k, 1] + k
            #                 if -1 < y and x < B.shape[0]:
            #                     average += B[y, x, :]
            #                     num += 1

            # if num > 0: image[mask_y0 + j, mask_x0 + i, :] = average / num
